# Stockout Risk Prediction

This notebook builds a model to predict whether a SKU will
stock out tomorrow using historical sales, availability,
and promotion data.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb


In [2]:
df = pd.read_csv("grocery_sales_autumn_2025.csv")
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values(["venue_id","sku_id", "date"])

df.head()


,date,venue_id,sku_id,phl1_id,phl2_id,phl3_id,operating_minutes,country_id,price,promo_flag,promo_depth,in_stock_minutes,stockout_flag,units_sold
0,2025-09-01,V001,S0001,P1_01,P2_03,P3_009,900,FI,0.55,0,0.0,900,0,67
450,2025-09-02,V001,S0001,P1_01,P2_03,P3_009,900,FI,0.59,0,0.0,900,0,16
900,2025-09-03,V001,S0001,P1_01,P2_03,P3_009,900,FI,0.59,0,0.0,900,0,28
1350,2025-09-04,V001,S0001,P1_01,P2_03,P3_009,900,FI,0.59,0,0.0,900,0,43
1800,2025-09-05,V001,S0001,P1_01,P2_03,P3_009,900,FI,0.59,0,0.0,900,0,15


In [3]:
df["target_stockout"] = (
    df.groupby(["venue_id","sku_id"])["stockout_flag"]
      .shift(-1))


In [4]:
df = df.dropna(subset=["target_stockout"])
df["target_stockout"] = df["target_stockout"].astype(int)

df[["date", "venue_id", "sku_id", "stockout_flag", "target_stockout"]].head(10)

,date,venue_id,sku_id,stockout_flag,target_stockout
0,2025-09-01,V001,S0001,0,0
450,2025-09-02,V001,S0001,0,0
900,2025-09-03,V001,S0001,0,0
1350,2025-09-04,V001,S0001,0,0
1800,2025-09-05,V001,S0001,0,0
2250,2025-09-06,V001,S0001,0,1
2700,2025-09-07,V001,S0001,1,0
3150,2025-09-08,V001,S0001,0,0
3600,2025-09-09,V001,S0001,0,0
4050,2025-09-10,V001,S0001,0,0


In [5]:
df["target_stockout"].value_counts(normalize=True)


target_stockout
0    0.62279
1    0.37721
Name: proportion, dtype: float64

In [6]:
df["in_stock_ratio"] = (
    df["in_stock_minutes"] / df["operating_minutes"])


In [7]:
df[["in_stock_minutes","operating_minutes", "in_stock_ratio"]].head(10)


,in_stock_minutes,operating_minutes,in_stock_ratio
0,900,900,1.0
450,900,900,1.0
900,900,900,1.0
1350,900,900,1.0
1800,900,900,1.0
2250,900,900,1.0
2700,0,900,0.0
3150,900,900,1.0
3600,900,900,1.0
4050,900,900,1.0


In [8]:
df.groupby("target_stockout")["in_stock_ratio"].mean()

target_stockout
0    0.716388
1    0.522325
Name: in_stock_ratio, dtype: float64

In [9]:
grp = df.groupby(["venue_id", "sku_id"])

df["sales_lag_1"]= grp["units_sold"].shift(1)
df["sales_7d_mean"] = grp["units_sold"].shift(1).rolling(7).mean()
df["sales_7d_max"] =grp["units_sold"].shift(1).rolling(7).max()


In [10]:
df = df.dropna(subset=["sales_lag_1", "sales_7d_mean", "sales_7d_max"]).copy()


In [11]:
df[["units_sold", "sales_lag_1","sales_7d_mean", "sales_7d_max"]].head(10)


,units_sold,sales_lag_1,sales_7d_mean,sales_7d_max
3150,46,0.0,27.714286,67.0
3600,38,46.0,24.714286,46.0
4050,12,38.0,27.857143,46.0
4500,26,12.0,25.571429,46.0
4950,61,26.0,23.142857,46.0
5400,36,61.0,29.714286,61.0
5850,0,36.0,31.285714,61.0
6300,41,0.0,31.285714,61.0
6750,37,41.0,30.571429,61.0
7200,27,37.0,30.428571,61.0


In [12]:
df.groupby("target_stockout")["sales_7d_mean"].mean()


target_stockout
0    8.741207
1    2.584151
Name: sales_7d_mean, dtype: float64

In [13]:
df["stockout_lag_1"] = grp["stockout_flag"].shift(1)
df["stockout_7d_sum"] = grp["stockout_flag"].shift(1).rolling(7).sum()


In [14]:
df = df.dropna(subset=["stockout_lag_1", "stockout_7d_sum"]).copy()


In [15]:
df.groupby("target_stockout")["stockout_7d_sum"].mean()


target_stockout
0    2.057013
1    3.564079
Name: stockout_7d_sum, dtype: float64

In [16]:
df["day_of_week"]= df["date"].dt.weekday
df["is_weekend"] =df["day_of_week"].isin([5, 6]).astype(int)



In [17]:
df["promo_depth"]= df["promo_depth"].fillna(0)


In [18]:
categorical_cols = [
    "phl1_id",
    "phl2_id",
    "phl3_id",
    "venue_id",
    "country_id",
]

for col in categorical_cols:
    df[col] = df[col].astype("category").cat.codes


In [19]:
df[categorical_cols].dtypes


phl1_id       int8
phl2_id       int8
phl3_id       int8
venue_id      int8
country_id    int8
dtype: object

In [20]:
split_date = df["date"].quantile(0.8)

train = df[df["date"] < split_date]
valid = df[df["date"] >= split_date]


In [21]:
features = [
    "in_stock_ratio",
    "sales_lag_1",
    "sales_7d_mean",
    "sales_7d_max",
    "stockout_lag_1",
    "stockout_7d_sum",
    "promo_flag",
    "promo_depth",
    "price",
    "day_of_week",
    "is_weekend",
    "phl1_id",
    "phl2_id",
    "phl3_id",
    "venue_id",
    "country_id",
]



In [22]:
X_train = train[features]
y_train = train["target_stockout"]

X_valid = valid[features]
y_valid = valid["target_stockout"]

X_train.shape, X_valid.shape


((29700, 16), (7650, 16))

In [23]:


model = lgb.LGBMClassifier(
    objective="binary",
    n_estimators=300,
    learning_rate=0.05,
    class_weight="balanced",
    random_state=42,
)

model.fit(X_train, y_train);



[LightGBM] [Info] Number of positive: 10974, number of negative: 18726
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000643 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1031
[LightGBM] [Info] Number of data points in the train set: 29700, number of used features: 15
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


In [24]:
valid = valid.copy()
valid["pred_proba"] = model.predict_proba(X_valid)[:, 1]

valid[["pred_proba", "target_stockout"]].head()



,pred_proba,target_stockout
32850,0.000145,0
33300,0.000294,0
33750,0.999721,1
34200,0.000282,0
34650,0.000118,0


In [25]:
def recall_at_k(y_true, y_score, k=0.05):
    cutoff=int(len(y_score)*k)
    top_k_idx=y_score.sort_values(ascending=False).index[:cutoff]
    return y_true.loc[top_k_idx].sum()/y_true.sum()

recall_5pct =recall_at_k(y_valid, valid["pred_proba"], k=0.05)
recall_10pct = recall_at_k(y_valid, valid["pred_proba"], k=0.10)

recall_5pct, recall_10pct



(np.float64(0.12251443232841565), np.float64(0.245349583066068))

In [26]:
baseline_score = valid["stockout_7d_sum"]

baseline_recall_5pct= recall_at_k(y_valid, baseline_score, k=0.05)
baseline_recall_10pct = recall_at_k(y_valid, baseline_score, k=0.10)

baseline_recall_5pct, baseline_recall_10pct



(np.float64(0.09781911481719051), np.float64(0.1895445798588839))